# Imports

In [1]:
#export
import os
import time
import yaml
import json
import datetime
import logging
import numpy as np
from pathlib import Path
from collections import namedtuple
import matplotlib.pyplot as plt
#%matplotlib inline

# Code

In [46]:
#export
def from_range(start, end, num, dist=None):
    if dist is None:
        dist = np.random.random
    lstart, lend = np.log(start), np.log(end)
    ar = np.logspace(lstart, lend, num, base=np.e)
    return ar

def get_dist(start, end, num, space, to_int=False):
    if space == 'log':
        space = np.geomspace
    elif space == 'lin':
        space = np.linspace
    else:
        raise NotImplementedError
    arr = space(start, end, num).astype(np.float32)
    if to_int:
        arr = arr.astype(np.int32)
    return arr

def dump_state(state, path, name):
    timestamp = '{:%Y_%b_%d_%H_%M_%S}'.format(datetime.datetime.now())
    p = path/f'{timestamp}_{name}.yaml'
    with open(p, 'w') as f:
        f.write(yaml.safe_dump(state, indent=4))
    return p

def init_params(raw_params):
    params = []
    for kwargs in raw_params:
        param = Param(**kwargs)
        sampling = kwargs['sampling']
        assert sampling == 'random' or sampling == 'sequential', sampling
        params.append({'sampling':sampling, 'instance':param}) 
    return params

In [47]:
#export
class Param:
    def __init__(self, name, arr, **kwargs):
        self.name = name
        self.arr = arr        
        self.count = 0
        
    def __len__(self):
        return len(self.arr)
    
    def __getitem__(self, i):
        i = i%self.__len__()
        return self.arr[i]
    
    def get_random(self):
        return self.__getitem__(np.random.choice(self.__len__()))
    
    def safe_get_next(self):
        val = self.__getitem__(self.count)
        self.count = self.count+1 if self.count < self.__len__()-1 else 0
        return val
    
    def get_next(self):
        if self.count >= self.__len__():
            raise StopIteration
        val = self.__getitem__(self.count)
        self.count += 1    
        return val
    
    def reset_count(self):
        self.count=0
    
    def __repr__(self):
        return f'Param : {self.name} | {str(self.arr)}'
    
    def plot(self):
        plt.hist(self.arr, bins=10)
    

In [73]:
#export
class BaseConfigCycler:        
    def get_values(self, params_dist, idx):
        values = {}
        for param in params_dist:
            sampling = param['sampling']
            if sampling == 'random':
                val = param['instance'].get_random()
            else:#elif sampling == 'sequential':
                if idx is not None:
                    val = param['instance'].__getitem__(idx)
                else:
                    val = param['instance'].safe_get_next()
            
            values[param['instance'].name] = val           
        return values
    
    def init_map(self):
        raise NotImplementedError
    
    def create_state(self, params_dist, idx=None):
        new_params = self.get_values(params_dist, idx)
        params_map = self.init_map()
        cfg = {}
        for name, (full_name, p_type, default_value) in params_map.items():
            value = new_params.get(name, default_value)
            if value is not np.NaN:
                cfg[full_name] = p_type(value)
        return cfg

In [74]:
class Cycler(BaseConfigCycler):
    def init_map(self):
        return {
            'g':('generations', int, 1),
            
            'e':('exp_power', int, 10),
            'f0':('dec_f0', int, np.NaN),
            'f1':('dec_f1', int, np.NaN),
            'f2':('dec_f2', int, np.NaN),
            'f3':('dec_f3', int, np.NaN),
            'mc':('mutate_chance', float, 0.003),
            'cc':('combine_chance', float, .6),
            'cr':('crossover_chance', float, .5),
            'co':('combine_chance', float, .8)
        }

In [75]:
p1 = {'name':'mc', 'arr':[.1,.2,.3], 'sampling':'sequential'}
p2 = {'name':'co', 'arr':[4,5,6], 'sampling':'random'}
p3 = {'name':'f0', 'arr':[33,488,99], 'sampling':'sequential'}
pis = [p1,p2, p3]
params_dist = init_params(pis)
c = Cycler()
c.create_state(params_dist=params_dist)

{'generations': 1,
 'exp_power': 10,
 'dec_f0': 33,
 'mutate_chance': 0.1,
 'combine_chance': 6.0,
 'crossover_chance': 0.5}

In [81]:
c.create_state(params_dist=params_dist)

{'generations': 1,
 'exp_power': 10,
 'dec_f0': 33,
 'mutate_chance': 0.1,
 'combine_chance': 5.0,
 'crossover_chance': 0.5}

In [101]:
c.create_state(params_dist=params_dist, idx=0)

{'generations': 1,
 'exp_power': 10,
 'dec_f0': 33,
 'mutate_chance': 0.1,
 'combine_chance': 4.0,
 'crossover_chance': 0.5}

# Export

In [103]:
!python3 n2s.py cycle.ipynb

Converted cycle.ipynb to exp/nb_cycle.py
